In [2]:
import ee, datetime
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
geometry = ee.Geometry.Polygon([[[77.42117255516425,23.20909792227606],[77.42057174034491,23.20799353262676],[77.42040007896796,23.207520219982936],[77.4200996715583,23.2069285768207],[77.41949885673895,23.207125791499145],[77.41902678795233,23.207007462726995],[77.41889804191962,23.206258044736966],[77.41868346519843,23.20550862254438],[77.41868346519843,23.204995857570196],[77.41851180382147,23.204680308915954],[77.41816848106757,23.203970321719954],[77.41885512657538,23.20314199855786],[77.41928428001776,23.202787001345527],[77.4204429943122,23.201840337503217],[77.42082923241034,23.201406447668166],[77.42125838585272,23.20089366695645],[77.42173045463934,23.20101200114144],[77.4224600154914,23.20140644766817],[77.42327540703192,23.20152478139923],[77.42361872978583,23.20187978196384],[77.42366164513007,23.202313670262438],[77.42413371391669,23.202589780264603],[77.42464869804755,23.203209392314182],[77.42507785148993,23.203761608047508],[77.42524951286688,23.20451104003633],[77.42451995201483,23.20569434514694],[77.42387622185126,23.206207107441198],[77.42348998375311,23.20707485453413],[77.42306083031075,23.208179251775938],[77.42284625358954,23.208692004535596],[77.42207377739327,23.20853423466525],[77.42143004722968,23.20837646460864],[77.42117255516425,23.20909792227606]]])

In [5]:
def get_chlorophyll(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)
# NDCI calculation
    def calculate_NDCI(image):
        ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
        return image.addBands(ndci)
    ndci = sentinel_AOI.map(calculate_NDCI)  

# NDCI mean
    def calculate_mean_NDCI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
        return mean.get('NDCI').getInfo()    
    
# NDCI mean collection    
    Images_ndci = ndci.select('NDCI').toList(ndci.size())
    ndci_coll = []
    for i in range(Images_ndci.length().getInfo()):
        image = ee.Image(Images_ndci.get(i-1))
        temp_ndci = calculate_mean_NDCI(image)
        ndci_coll.append(temp_ndci)
# Dates collection
    dates = np.array(ndci.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for chlorophyll
    df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df


In [6]:
def get_turbidity(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

#calculate NDTI
    def calculate_NDTI(image):
        ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
        return image.addBands(ndti)
    ndti = sentinel_AOI.map(calculate_NDTI)

# Mean NDTI
    def calculate_mean_NDTI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('NDTI').getInfo()
    
# NDTI mean collection 
    Images_ndti = ndti.select('NDTI').toList(ndti.size())
    ndti_coll = []
    for i in range(Images_ndti.length().getInfo()):
        image = ee.Image(Images_ndti.get(i-1))
        temp_ndti = calculate_mean_NDTI(image)
        ndti_coll.append(temp_ndti)

# Dates Collection
    dates = np.array(ndti.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for Turtbidity

    df = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [7]:
def get_Salanity(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

#calculate NDSI
    def calculate_NDSI(image):
        ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
        return image.addBands(ndsi)
    ndsi = sentinel_AOI.map(calculate_NDSI)

# Mean NDSI
    def calculate_mean_NDSI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('NDSI').getInfo()
        
# NDSI Mean Collection
    Images_ndsi = ndsi.select('NDSI').toList(ndsi.size())
    ndsi_coll = []
    for i in range(Images_ndsi.length().getInfo()):
        image = ee.Image(Images_ndsi.get(i-1))
        temp_ndsi = calculate_mean_NDSI(image)
        ndsi_coll.append(temp_ndsi)

# Dates Collection
    dates = np.array(ndsi.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for Salinity

    df = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinity'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [8]:
def get_DO(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

# calculate Dissolved Oxygen
    def calculate_DO(image):
        do = image.normalizedDifference(['B4', 'B5']).rename('DO')
        return image.addBands(do)
    do = sentinel_AOI.map(calculate_DO)

# Mean DO

    def calcualte_mean_DO(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                            geometry = AOI,
                            scale = image.projection().nominalScale().getInfo(),
                            maxPixels = 100000,
                            bestEffort = True);
        return mean.get('DO').getInfo()
        
# DO Mean Collection

    Images_do = do.select('DO').toList(do.size())
    do_coll = []
    for i in range(Images_do.length().getInfo()):
        image = ee.Image(Images_do.get(i-1))
        temp_do = calcualte_mean_DO(image)
        do_coll.append(temp_do)

# Dates Collection
    dates = np.array(do.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for DO

    df = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [9]:
def get_pH(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

# calculate pH

    def calculate_pH(image):
        ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
        return image.addBands(ph)
    pH = sentinel_AOI.map(calculate_pH)

# Mean pH

    def calculate_mean_pH(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('PH').getInfo()
        
# pH Mean Collection

    Images_ph = pH.select('PH').toList(pH.size())
    ph_coll= []
    for i in range(Images_ph.length().getInfo()):
        image = ee.Image(Images_ph.get(i-1))
        temp_ph = calculate_mean_pH(image)
        ph_coll.append(temp_ph)

# Dates Collection

    dates = np.array(pH.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for pH

    df = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [10]:
def get_Temprature(start_date, end_date):

# Selecting the satellite and AOI  
# Landsat 8 
# LC08/C02/T1_L2
    landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate(start_date,end_date)
    AOI = geometry
    landsat_AOI = landsat.filterBounds(AOI)

# calculate Temperature

    def calculate_temp(image):
        temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
        return image.addBands(temp)
    temp = landsat_AOI.map(calculate_temp)

# Mean Temperature

    def calculate_mean_temp(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('temperature').getInfo()
    
# Temprature Mean Collection

    Images_temp = temp.select('temperature').toList(temp.size())
    temp_coll = []
    for i in range(Images_temp.length().getInfo()):
        image = ee.Image(Images_temp.get(i-1))
        tempe_temp = calculate_mean_temp(image)
        temp_coll.append(tempe_temp)

# Dates Collection

    dates = np.array(temp.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for temperature

    df = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [11]:
def get_DOM(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 3 OLCI
# Copernicus Sentinel-3 OLCI 
    sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate(start_date, end_date)
    AOI = geometry

    sentinel3_AOI = sentinel3.filterBounds(AOI)

# calculate Dissolved Organic Matter (DOM) from Sentinel-3 OLCI

    def calculate_DM(image):
        rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
        dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
        return image.addBands(dm)
    dm = sentinel3_AOI.map(calculate_DM)

# Mean DOM

    def calcualte_mean_DM(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                            geometry = AOI,
                            scale = image.projection().nominalScale().getInfo(),
                            maxPixels = 100000,
                            bestEffort = True);
        return mean.get('dom').getInfo()
    
# DOM Mean Collection

    Images_dm = dm.select('dom').toList(dm.size())
    dm_coll= []
    for i in range(Images_dm.length().getInfo()):
        image = ee.Image(Images_dm.get(i-1))
        temp_dm = calcualte_mean_DM(image)
        dm_coll.append(temp_dm)

# Dates Collection

    dates = np.array(dm.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for DOM

    df = pd.DataFrame(dm_coll, index = day, columns = ['dom'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [12]:
def get_sm(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 3 OLCI
# Copernicus Sentinel-3 OLCI 
    sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate(start_date, end_date)
    AOI = geometry

    sentinel3_AOI = sentinel3.filterBounds(AOI)

# calculate suspended matter

    def calculate_SM(image):
        rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
        suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
        return image.addBands(suspended_matter)
    sm = sentinel3_AOI.map(calculate_SM)

# Mean of Suspended Matter

    def meanSM(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
        return mean.get('suspended_matter').getInfo()
            
# Suspended-Matter Mean Collection

    Images_sm = sm.select('suspended_matter').toList(sm.size())
    sm_coll= []
    for i in range(Images_sm.length().getInfo()):
        image = ee.Image(Images_sm.get(i-1))
        temp_sm = meanSM(image)
        sm_coll.append(temp_sm)

# Dates Collection

    dates = np.array(sm.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for suspended matter

    df = pd.DataFrame(sm_coll, index = day, columns = ['Suspended Matter'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [13]:
'''df1 = get_chlorophyll('2019-01-01', '2019-01-10')
   df1.to_csv('chlorophyll.csv')'''

#In this format, the data is stored in CSV.


"df1 = get_chlorophyll('2019-01-01', '2019-01-10')\n   df1.to_csv('chlorophyll.csv')"

In [15]:
# chlorophyll = get_chlorophyll('2019-01-01', '2019-01-10')
# turbidity = get_turbidity('2019-01-01', '2019-01-10')
# temperature = get_temperature('2019-01-01', '2019-01-10')
# DOM = get_DOM('2019-01-01', '2019-01-10')
# suspended_matter = get_sm('2019-01-01', '2019-01-10')
# pH = get_pH('2019-01-01', '2019-01-10')
# dissolved_oxygen = get_DO('2019-01-01', '2019-01-10')